In [73]:
using Combinatorics
using LinearAlgebra

In [3]:
function gaussian_elimination(A, U)
    m, n = size(A)
    # Create the augmented matrix by concatenating A and U
    augmented_matrix = hcat(A, U)

    for j in 1:n
        # Find the pivot row
        pivot_row = nothing
        for k in j:m
            if augmented_matrix[k, j]!= 0
                pivot_row = k
                break
            end
        end

        if pivot_row === nothing
            # No pivot in this column, skip to the next column
            continue
        end

        # Swap current row with pivot row if needed
        if pivot_row!= j
            augmented_matrix[[j, pivot_row], :] = augmented_matrix[[pivot_row, j], :]
        end

        # Normalize the pivot row
        augmented_matrix[j, :]./= augmented_matrix[j, j]

        # Eliminate the current column in all other rows
        for i in 1:m
            if i!= j
                augmented_matrix[i, :] -= augmented_matrix[i, j] * augmented_matrix[j, :]
            end
        end
    end

    # Check for inconsistency
    for i in (n+1):m
        if augmented_matrix[i, end]!= 0
            return nothing  # No solution
        end
    end

    return augmented_matrix[1:n, end]
end

A = [2.0 1.0 5.0; 1.0 3.0 6.0; 0.0 4.0 3.0]
U = [1.0; 2.0; 3.0]
result = gaussian_elimination(A, U)
println("Solution: ", result)

Solution: [0.6153846153846155, 0.9230769230769231, -0.23076923076923084]


In [61]:
function Cnk(x, y)
    return collect(combinations(y, x))
end

function matrix_u(x, s)
    return x[:, s]
end

function inverse(x)
    return inv(x)
end

function supp(x)
    return findall(x -> x != 0, x)
end

function nnz(matrix)
    m, n = size(matrix)
    zero_count = 0
    for row in eachrow(matrix)
        for element in row
            if element == 0 || abs(element) <= 1e-16
                zero_count += 1
            end
        end
    end
    return m * n - zero_count
end


nnz (generic function with 1 method)

In [36]:
function omega_validator3(u)
    a = [i for i in 1:size(u, 1) if !(i in omega)]
    for i in a
        u_without_ith_row = u[setdiff(1:size(u, 1), [i]), :]
        if rank(u_without_ith_row) == size(u, 1) - 1
            result = u[i, :]' * inv(u_without_ith_row)
            return vcat(result[1:i-1], -1.0, result[i:end])
        end
    end
    return nothing
end


function omega_validator4(u::Matrix{Float64})
    a = [i for i in 1:size(u, 1) if !(i in omega)]

    for i in a
        u_without_ith_row = u[setdiff(1:end, i), :]
        if rank(u_without_ith_row) == size(u, 1) - 1
            result = gaussian_elimination(u_without_ith_row', u[i, :])
            if result !== nothing
                return vcat(result[1:i-1], -1.0, result[i:end])
            end
        end
    end
    return nothing
end


omega_validator4 (generic function with 1 method)

In [64]:
function algorithm3(u)
    sparsity, sparsest, i = 0, nothing, nothing

    for c in preprocessed
        Uc = matrix_u(u, collect(c))
        t = omega_validator3(Uc)

        if t !== nothing
            v = t' * u
            E = [i for i in v if i == 0]

            if length(E) > sparsity
                sparsity, sparsest = length(E), v
                i = filter(i -> !(i in omega), supp(t))[1]
            end
        end
    end

    return sparsest, i
end

function algorithm4(u)
    n, m = size(u)
    a = [i for i in m-1:-1:n-1]
    for z in a
        for c in Cnk(z, [i for i in 1:m])
            Uc = matrix_u(u, collect(c))
            if omega_validator4(Uc) !== nothing
                t = omega_validator4(Uc)
                v = t' * u
                i = filter(i -> !(i in omega), supp(t))[1]
                return v, i
            end
        end
    end
    return nothing,nothing
end

using LinearAlgebra

function algorithm5(u)
    m, n = size(u)
    sparsity, basis = nnz(u), Matrix{Float64}(I, n, n)
    for c in Cnk(m, collect(1:n))
        Uc = matrix_u(u, c)
        if rank(Uc) == m
            sparsifier = inv(Uc)
            print(sparsifier)
            if nnz(sparsifier * u) < sparsity
                sparsity, basis = nnz(sparsifier * u), sparsifier
            end
        end
    end
    return basis
end



algorithm5 (generic function with 2 methods)

In [48]:
function algorithm2_3(u)
    global omega, preprocessed

    omega = []
    preprocessed = [j for j in Cnk(size(u, 1) - 1, [i for i in 1:size(u, 2)]) if rank(matrix_u(u, collect(j))) == size(u, 1) - 1]

    for j in 1:length(u)
        v, i = algorithm3(u)
        if i !== nothing
            u[i, :] = v
            push!(omega, i)
        end
    end

    return u
end

function algorithm2_4(u)
    global omega

    omega = []
    for j in 1:length(u)
        v, i = algorithm4(u)
        if i !== nothing
            u[i, :] = v
            push!(omega, i)
        end
    end

    return u
end

algorithm2_4 (generic function with 1 method)

In [69]:
# Define matrices
example0 = [4/9 -8/9 -8/9 -4/9; 
            0 5/9 0 10/9; 
            8/9 -2/3 0 0; 
            4/9 2/9 8/9 4/9; 
            0 -10/9 0 0; 
            4/9 -1/3 -8/9 2/3; 
            -4/9 -2/9 8/9 4/9]

example1 = copy(example0) # Creating a copy of example0
example2=copy(example0)
b = [8/9 -2/3 0 0; 
     0 5/9 0 10/9; 
     0 -10/9 0 0; 
     4/9 -8/9 -8/9 -4/9]

c = [0 0 0 1; 
     0 1 0 0; 
     1 0 0 0; 
     1 0 0 -1; 
     0 0 1 0; 
     0 1 0 1; 
     0 0 1 -1]

# Perform matrix multiplication
result = c * b

println(result)


[0.4444444444444444 -0.8888888888888888 -0.8888888888888888 -0.4444444444444444; 0.0 0.5555555555555556 0.0 1.1111111111111112; 0.8888888888888888 -0.6666666666666666 0.0 0.0; 0.4444444444444444 0.2222222222222222 0.8888888888888888 0.4444444444444444; 0.0 -1.1111111111111112 0.0 0.0; 0.4444444444444444 -0.33333333333333326 -0.8888888888888888 0.6666666666666667; -0.4444444444444444 -0.22222222222222232 0.8888888888888888 0.4444444444444444]


In [72]:
example0_transposed = transpose(example0)
example1_transposed = transpose(example1)
example2_transposed = transpose(example2)
println("Exact Solutions")
println(algorithm2_3(example0_transposed))
println(algorithm2_4(example1_transposed))
println("heuristic Solutions")
println(algorithm5(example2_transposed))

Exact Solutions
[0.0 0.0 0.8888888888888888 0.8888888888888888 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -1.1111111111111112 0.0 -1.1111111111111112; 0.0 -2.2222222222222223 0.0 0.0 0.0 -2.2222222222222223 0.0; -0.4444444444444444 0.0 0.0 0.4444444444444444 0.0 -0.44444444444444453 0.4444444444444444]
[0.0 0.0 0.0 1.2335811384723967e-17 -1.1111111111111112 -7.395570986446989e-32 -1.1111111111111112; 0.3333333333333331 0.0 0.3333333333333332 3.700743415417185e-17 4.444444444444445 0.3333333333333331 4.111111111111112; 0.0 -2.2222222222222223 0.0 0.0 0.0 -2.2222222222222223 0.0; 0.0 0.0 0.44444444444444453 0.44444444444444453 0.0 -1.1102230246251565e-16 -2.220446049250313e-16]
heuristic Solutions
[1.4988010832439614e-16 7.494005416219807e-17 -1.125 0.0; 7.494005416219807e-17 9.298117831235686e-17 -0.45 0.8999999999999999; 1.125 0.0 0.5625 0.0; -0.675 -0.9 0.3375 0.45][-0.6749999999999999 -0.8999999999999999 -0.7875000000000001 0.44999999999999996; 7.494005416219807e-17 9.298117831235686e-17 -0.45 0.8